In [1]:
import json 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, concatenate, Dropout, GlobalAveragePooling1D
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification, AutoTokenizer, AutoModel, AutoModelForMaskedLM
import os
import numpy as np
from PIL import Image
import cv2
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from keras.saving import register_keras_serializable

In [2]:
!pip install google-cloud-bigquery[bqstorage,pandas]==3.10.0
!pip install google-cloud-storage>=2.0.0
!pip install pandas<2.1.4,>=1.5.0
!pip install numpy<3.0,>=1.20
!pip install pydantic~=1.10.0
!pip install albumentations==1.4.10
!pip install scipy<1.14,>=1.4.1

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.4/218.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.25.0
    Uninstalling google-cloud-bigquery-3.25.0:
      Successfully uninstalled google-cloud-bigquery-3.25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.17.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.16.0, but you have google-cloud-bigquery 3.10.0 which is incompatible.
bigquery-magics 0.2.0 requires google-cloud-bigquery<4.0.0dev,>=3.13.0, but you have google-cloud-bigquery 3.10.0 which is incompatible.
pandas-gbq 0.23.1 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
/bin/bash: line 1: 2.1.4,: No such file or director

In [3]:
!pip install -q paddleocr paddlepaddle
!pip install -q vietocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 7.0 MB/s 

In [4]:
!pip install einops 

In [5]:
!pip install --quiet scikit-learn==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 81.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.2 requires scikit-learn>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
bigframes 1.17.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.16.0, but you have google-cloud-bigquery 3.10.0 which is incompatible.
paddleocr 2.9.1 requires albumentations==1.4.10, but you have albumentations 1.4.2 which is incompatible.


In [6]:
from paddleocr import PaddleOCR
from vietocr.tool.config import Cfg
from vietocr.tool.predictor import Predictor

In [7]:
def read_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
        data_list = [{**value} for key, value in data.items()]
        
        df = pd.DataFrame(data_list)
        
    return df

In [8]:
def read_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
        data_list = [{**value} for key, value in data.items()]
        
        df = pd.DataFrame(data_list)
        
    return df

In [9]:
path = '/kaggle/input/data-ai002/data_AI002/data_AI002/'

In [10]:
json_path = '/kaggle/input/data-ai002/data-AI002.json'

In [11]:
df = read_json(json_path)
df

,image,caption,label
0,8ae451edcd8ebf697f8763ece249115813149c55733bf8...,Cô ấy trên mạng vs cô ấy ngoài đời =))),sarcasm
1,35370ffd6c791d6f8c4ab3dd4363ed468fab41e4824ee9...,Người tâm linh giao tiếp với người thực tế :))),sarcasm
2,316fdd1477725b9fb1a55015ac06b68b92b50bd4303e08...,Hình như Trăng hôm nay đẹp quá mọi người ạ! 😃 ...,sarcasm
3,8a0f34e0e30e4e5cfb306933c1d25fa801a5da78646b59...,MỌI NGƯỜI NGHĨ SAO VỀ PHÁT BIỂU CỦA SHARK VIỆT...,not-sarcasm
4,e517a5e95d1065886a7c815e82fe254381d4f9f4b244d4...,2 tay hai nàng chứ việc gì phải lệ hai hàng,sarcasm
...,...,...,...
11902,3c643826258f8aacc8a98d8e24956f909797010f1e80bd...,Mẹ biết mẹ buồn ó 🐧,sarcasm
11903,09f5adf3e555d3066eae0be356e5ce797c066706e7c808...,Ủa ăn đi anh sao tự nhiên rén vậy? \n#KFCViet...,sarcasm
11904,b11515c1aa521da4f6d0cd6464e5ea3e030662cdd43f7f...,Mẹ không phát hiện sớm là vài tháng nữa con Ch...,sarcasm
11905,8a83634808704a5c7493327893f793effd6e78cb037ac4...,Anh em h.út ch.ích gánh còng lưng :(((,sarcasm


In [12]:
class OCRProcessor:
    def __init__(self):
        # Initialize PaddleOCR and VietOCR models
        self.paddle_ocr = PaddleOCR(use_angle_cls=True, lang='vi', use_gpu=True)
        
        # Load VietOCR model with configuration
        config = Cfg.load_config_from_name('vgg_transformer')
        config['weights'] = '/kaggle/input/weights-vgg-transformer/pytorch/default/1/vgg_transformer.pth'
        config['cnn']['pretrained'] = True
        config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.vietocr_model = Predictor(config)

    def merge_boxes(self, boxes, threshold=23):
        """Gộp các bounding boxes nếu chúng quá gần nhau (dưới threshold)."""
        merged_boxes = []
        boxes = sorted(boxes, key=lambda x: x[0])  # Sort by x_min
        
        while boxes:
            current_box = boxes.pop(0)
            x_min, y_min, x_max, y_max = current_box
            merged = False
            
            for i, box in enumerate(merged_boxes):
                bx_min, by_min, bx_max, by_max = box
                # Merge boxes if they are close enough
                if abs(x_min - bx_max) < threshold and abs(y_min - by_max) < threshold:
                    merged_boxes[i] = [min(x_min, bx_min), min(y_min, by_min), max(x_max, bx_max), max(y_max, by_max)]
                    merged = True
                    break
                    
            if not merged:
                merged_boxes.append(current_box)
        
        return merged_boxes

    def process_images(self, directory, output_csv_file_path="/kaggle/working/ocr_text.csv"):
        # Prepare the dataframe for storing results
        df = pd.DataFrame(columns=["image_name", "combined_text"])
        total_images = len(os.listdir(directory))

        for i, filename in enumerate(os.listdir(directory), 1):
            image_path = os.path.join(directory, filename)
            
            # Skip non-image files
            if not os.path.isfile(image_path) or not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                continue
            
            print(f"Processing image {i}/{total_images} ({filename})", end='\r')
            
            combined_text = None
            try:
                # Perform OCR using PaddleOCR
                ocr_results = self.paddle_ocr.ocr(image_path, cls=True)
                recognized_texts = []
                boxes = []
                
                for line in ocr_results[0]:  # ocr_results[0] contains the list of lines
                    if len(line) == 2:  # bbox and text
                        bbox, text = line
                        confidence = None
                    elif len(line) == 3:  # bbox, text, and confidence
                        bbox, text, confidence = line
                    else:
                        continue
                    
                    # Store bounding boxes and recognized texts
                    boxes.append([bbox[0][0], bbox[0][1], bbox[2][0], bbox[2][1]])
                    recognized_texts.append(text)
                
                # Merge close bounding boxes
                merged_boxes = self.merge_boxes(boxes)
                recognized_texts = []  # Store recognized texts after merging bounding boxes
                
                for box in merged_boxes:
                    x_min, y_min, x_max, y_max = map(int, box)
                    cropped_region = Image.open(image_path).crop((x_min, y_min, x_max, y_max))
                    recognized_text = self.vietocr_model.predict(cropped_region)
                    recognized_texts.append(recognized_text)
                
                # Combine the recognized text
                combined_text = "\n".join(recognized_texts)
            
            except Exception as ocr_error:
                print(f"\nError extracting OCR text for image {filename}: {str(ocr_error)}")
                combined_text = ""  # If OCR fails, save empty text
            
            # Add the OCR results to the dataframe
            new_row = pd.DataFrame({"image_name": [filename], "combined_text": [combined_text]})
            df = pd.concat([df, new_row], ignore_index=True)
        
        # Save results to CSV
        df.to_csv(output_csv_file_path, index=False, encoding="utf-8")
        print(f"\nOCR results saved to {output_csv_file_path}")
        return df

In [13]:
ocr_processor = OCRProcessor()

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:16<00:00, 234.22it/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/latin_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer/latin_PP-OCRv3_rec_infer.tar


100%|██████████| 9930/9930 [00:41<00:00, 237.39it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:08<00:00, 238.89it/s]

[2024/12/26 12:06:42] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25,


Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:02<00:00, 217MB/s]


In [14]:
output_csv_file_path = "/kaggle/working/ocr_text.csv"
ocr_processor.process_images(path, output_csv_file_path)

[2024/12/26 12:06:53] ppocr DEBUG: dt_boxes num : 10, elapsed : 0.16376280784606934
[2024/12/26 12:06:53] ppocr DEBUG: cls num  : 10, elapsed : 0.05354881286621094
[2024/12/26 12:06:53] ppocr DEBUG: rec_res num  : 10, elapsed : 0.8275895118713379
[2024/12/26 12:06:56] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.18778014183044434
[2024/12/26 12:06:56] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/12/26 12:06:56] ppocr DEBUG: rec_res num  : 0, elapsed : 1.9073486328125e-06

Error extracting OCR text for image 733ef474a9c5a33c867728388aaf3f7247c90b34a86dec7ebb4bede260275871.jpg: 'NoneType' object is not iterable
[2024/12/26 12:06:56] ppocr DEBUG: dt_boxes num : 8, elapsed : 0.054033517837524414
[2024/12/26 12:06:56] ppocr DEBUG: cls num  : 8, elapsed : 0.03473782539367676
[2024/12/26 12:06:57] ppocr DEBUG: rec_res num  : 8, elapsed : 0.8709044456481934
[2024/12/26 12:06:58] ppocr DEBUG: dt_boxes num : 6, elapsed : 0.05144000053405762
[2024/12/26 12:06:58] ppocr DEBUG: cls num  : 6, elapsed 

,image_name,combined_text
0,98c78da23f651d72af4ab7100d0f826fde43704c8273e7...,E\nfacebook com/trollbongda\nNHƯNG CHỈ DÙ HAY ...
1,733ef474a9c5a33c867728388aaf3f7247c90b34a86dec...,
2,9f32b0eee517243bf80a0b1f9c9e52a44af9589135360a...,NẾU TRẢ LÀ DÙNG BAN XỨNG ĐẢNG LÀ NHÀ THỊ\nNẾU ...
3,c83c7c3f5936299e3d9f20952042e6c4454d8105b54683...,stercard\nfacebook com/trollbongda\nCHỈ CÓ THỂ...
4,e32b566d3f41372526cc59d3a47ecbe4d72a38817416eb...,
...,...,...
12304,c1c1b4c07aa9224ddc1c949fac863523355d7545184302...,%TOP1:\nTOPNHÃN BIẾT NGƯỜI ĐĂNG THỊ\nTRƯỞNG THÀNH
12305,2b6b43705482d258ced6ff68ffc0deeafddd3a4a605a7f...,THUY QU.ÂY RỐI JIN (BTS) TRONG QUÂN NGỮ\nNỮ ĐI...
12306,3d552d33ee924fd7e66027f79c843ca460d97d7f8386f5...,SAU VIẾC TRẦN THÀNH ?CĂN SƯ RIÊNG TƯ?\nTHÁNG 2...
12307,8dea7a70d36a0bb00a558d87c76dbe46c192bbc253bd92...,facebook com/trollbongda\nNẾU DƯỚCXEM TRUCTIEP...
